In [ ]:
import nonlinear_benchmarks
import nonlinear_benchmarks.error_metrics as metrics
import jax.numpy as jnp
import torch
from jaxid.common import MLP
from jaxid.statespace import StateUpdateMLP, Simulator
import matplotlib.pyplot as plt

In [ ]:
#ckpt = torch.load("ss_subnet.pt")
ckpt = torch.load("ss.pt")
cfg = ckpt["cfg"]

In [ ]:
f_xu = StateUpdateMLP(cfg.hidden_f + [cfg.nx])
g_x = MLP(cfg.hidden_g + [cfg.ny])
model = Simulator(f_xu, g_x)

In [ ]:
# Load test data
train_val, test = nonlinear_benchmarks.WienerHammerBenchMark()
u_test, y_test = test
u_test = u_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
# Load scalers
params = ckpt["params"]
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

In [ ]:
u_test = scaler_u.transform(u_test)
x0 = jnp.zeros(cfg.nx)
y_test_hat = model.apply(params, x0, u_test)
y_test_hat = scaler_y.inverse_transform(y_test_hat)

In [ ]:
plt.plot(y_test, "k")
plt.plot(y_test_hat, "b")
plt.plot(y_test_hat - y_test, "r")
plt.show()

In [ ]:
plt.plot(ckpt["LOSS"])

In [ ]:
fit = metrics.fit_index(y_test, y_test_hat)[0]
rmse = metrics.RMSE(y_test, y_test_hat)[0] * 1000
nrmse = metrics.NRMSE(y_test, y_test_hat)[0]

print(f"{fit=} \n{rmse=} mV \n{nrmse=}")